In [1]:
import pyspark
import glow

In [2]:
builder = pyspark.sql.SparkSession.builder.appName("GlowVCFExplore") \
    .config("spark.jars.packages", "io.projectglow:glow-spark3_2.12:1.2.1") \
    .config("spark.hadoop.io.compression.codecs", "io.projectglow.sql.util.BGZFCodec")

In [3]:
spark = builder.getOrCreate()

In [4]:
spark = glow.register(spark)

In [ ]:
spark

In [ ]:
# Page Break

# Explore bcBio both Somatic and Germline VCFs

In [5]:
# Observe that somatic schema has 2 additional columns `INFO_FREQ` and `INFO_SOMTYPE`
!diff schema_bcbio_giab_somatic.txt schema_bcbio_giab_germline.txt

22,23d21
<  |-- INFO_FREQ: array (nullable = true)
<  |    |-- element: string (containsNull = true)
37,38d34
<  |-- INFO_SOMTYPE: array (nullable = true)
<  |    |-- element: string (containsNull = true)


In [6]:
# This time we just point to data directory
bcbio_src = "./data/bcbio_giab_somatic/*.vcf.gz"

In [ ]:
bcbio_df = spark.read.format("vcf").load(bcbio_src)

In [8]:
# Observe that Spark has merged the schema i.e. Union of columns from both VCF headers
bcbio_df.printSchema()

root
 |-- contigName: string (nullable = true)
 |-- start: long (nullable = true)
 |-- end: long (nullable = true)
 |-- names: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- referenceAllele: string (nullable = true)
 |-- alternateAlleles: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- qual: double (nullable = true)
 |-- filters: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- splitFromMultiAllelic: boolean (nullable = true)
 |-- INFO_platformnames: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- INFO_callsetwithotheruniqgenopassing: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- INFO_callsetnames: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- INFO_AC: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- INFO_FREQ: array (nullable = true)
 |    |-- element: string (containsNull = true)


In [ ]:
bcbio_df.createOrReplaceTempView("vcf_table")

In [10]:
spark.sql("describe vcf_table").show(n=1000, truncate=True)

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|          contigName|              string|   null|
|               start|              bigint|   null|
|                 end|              bigint|   null|
|               names|       array<string>|   null|
|     referenceAllele|              string|   null|
|    alternateAlleles|       array<string>|   null|
|                qual|              double|   null|
|             filters|       array<string>|   null|
|splitFromMultiAll...|             boolean|   null|
|  INFO_platformnames|       array<string>|   null|
|INFO_callsetwitho...|       array<string>|   null|
|   INFO_callsetnames|       array<string>|   null|
|             INFO_AC|          array<int>|   null|
|           INFO_FREQ|       array<string>|   null|
|        INFO_varType|              string|   null|
|          INFO_DPSum|                 int|   null|
|INFO_datase

In [11]:
# Observe that total variants count from both VCFs in single table
spark.sql("select count(1) as number_of_variants from vcf_table").show()

WARNING	2022-10-07 23:28:40	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:28:40	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:28:40	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:28:40	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:28:43	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream


+------------------+
|number_of_variants|
+------------------+
|           3206719|
+------------------+



In [12]:
spark.sql("select distinct contigName from vcf_table order by contigName").show(46)

WARNING	2022-10-07 23:28:49	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:28:49	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:28:49	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:28:49	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:28:52	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream


+----------+
|contigName|
+----------+
|      chr1|
|     chr10|
|     chr11|
|     chr12|
|     chr13|
|     chr14|
|     chr15|
|     chr16|
|     chr17|
|     chr18|
|     chr19|
|      chr2|
|     chr20|
|     chr21|
|     chr22|
|      chr3|
|      chr4|
|      chr5|
|      chr6|
|      chr7|
|      chr8|
|      chr9|
+----------+



In [13]:
spark.sql("select contigName, start, end from vcf_table").show()

+----------+------+------+
|contigName| start|   end|
+----------+------+------+
|      chr1|817185|817186|
|      chr1|817340|817341|
|      chr1|817888|817889|
|      chr1|818801|818802|
|      chr1|818811|818812|
|      chr1|818953|818954|
|      chr1|819122|819123|
|      chr1|819583|819584|
|      chr1|824319|824320|
|      chr1|824456|824457|
|      chr1|825531|825532|
|      chr1|825766|825767|
|      chr1|826576|826577|
|      chr1|826892|826893|
|      chr1|827208|827209|
|      chr1|827211|827212|
|      chr1|827220|827221|
|      chr1|827251|827252|
|      chr1|828013|828014|
|      chr1|830724|830725|
+----------+------+------+
only showing top 20 rows



WARNING	2022-10-07 23:28:58	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream


In [14]:
spark.sql("select contigName, count(end) as num_of_pos from vcf_table group by contigName order by num_of_pos desc").show(46)

WARNING	2022-10-07 23:29:02	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:29:02	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:29:02	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:29:02	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:29:04	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream


+----------+----------+
|contigName|num_of_pos|
+----------+----------+
|      chr2|    271356|
|      chr1|    267876|
|      chr3|    243088|
|      chr6|    220641|
|      chr4|    213774|
|      chr5|    204282|
|      chr7|    186041|
|     chr11|    172760|
|     chr10|    169586|
|      chr8|    165373|
|     chr12|    154561|
|      chr9|    147990|
|     chr13|    137088|
|     chr14|    112042|
|     chr15|     96779|
|     chr17|     83405|
|     chr18|     80893|
|     chr20|     71827|
|     chr19|     61776|
|     chr16|     60843|
|     chr21|     47651|
|     chr22|     37087|
+----------+----------+



In [15]:
spark.sql("select referenceAllele, alternateAlleles, array_size(alternateAlleles) from vcf_table").show()

+---------------+----------------+----------------------------+
|referenceAllele|alternateAlleles|array_size(alternateAlleles)|
+---------------+----------------+----------------------------+
|              G|             [A]|                           1|
|              A|             [G]|                           1|
|              C|             [G]|                           1|
|              A|             [G]|                           1|
|              A|             [G]|                           1|
|              T|             [C]|                           1|
|              G|             [A]|                           1|
|              C|             [T]|                           1|
|              T|             [C]|                           1|
|              T|             [A]|                           1|
|              C|             [T]|                           1|
|              T|             [C]|                           1|
|              A|            [AT]|      

WARNING	2022-10-07 23:29:10	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream


In [16]:
spark.sql("select referenceAllele, alternateAlleles, count(*) as num_of_snps \
from vcf_table \
where \
    char_length(referenceAllele) = 1 and \
    array_size(alternateAlleles) = 1 and \
    char_length(alternateAlleles[0]) = 1 \
    group by referenceAllele, alternateAlleles \
    order by num_of_snps desc").show()

WARNING	2022-10-07 23:29:14	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:29:14	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:29:14	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:29:14	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:29:16	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream


+---------------+----------------+-----------+
|referenceAllele|alternateAlleles|num_of_snps|
+---------------+----------------+-----------+
|              C|             [T]|     484436|
|              G|             [A]|     484020|
|              T|             [C]|     463525|
|              A|             [G]|     462312|
|              G|             [C]|     119203|
|              C|             [G]|     118972|
|              G|             [T]|     117604|
|              C|             [A]|     117027|
|              A|             [C]|     114401|
|              T|             [G]|     113450|
|              T|             [A]|      96640|
|              A|             [T]|      96202|
+---------------+----------------+-----------+



In [17]:
spark.sql("select contigName, start, end, referenceAllele, alternateAlleles, genotypes.sampleId, genotypes.alleleDepths \
from vcf_table").show(truncate=False)

+----------+------+------+---------------+----------------+---------+------------+
|contigName|start |end   |referenceAllele|alternateAlleles|sampleId |alleleDepths|
+----------+------+------+---------------+----------------+---------+------------+
|chr1      |817185|817186|G              |[A]             |[NA12878]|[[78, 454]] |
|chr1      |817340|817341|A              |[G]             |[NA12878]|[[107, 342]]|
|chr1      |817888|817889|C              |[G]             |[NA12878]|[[74, 220]] |
|chr1      |818801|818802|A              |[G]             |[NA12878]|[[0, 202]]  |
|chr1      |818811|818812|A              |[G]             |[NA12878]|[[0, 190]]  |
|chr1      |818953|818954|T              |[C]             |[NA12878]|[[0, 246]]  |
|chr1      |819122|819123|G              |[A]             |[NA12878]|[[110, 363]]|
|chr1      |819583|819584|C              |[T]             |[NA12878]|[[91, 378]] |
|chr1      |824319|824320|T              |[C]             |[NA12878]|[[92, 391]] |
|chr

WARNING	2022-10-07 23:29:26	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream


In [18]:
spark.sql("select contigName, start, end, referenceAllele, alternateAlleles, genotypes.sampleId from vcf_table \
where contigName = 'chr1' and end = 817186").show(truncate=False)

WARNING	2022-10-07 23:29:29	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:29:32	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:29:32	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:29:32	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:29:33	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream


+----------+------+------+---------------+----------------+---------+
|contigName|start |end   |referenceAllele|alternateAlleles|sampleId |
+----------+------+------+---------------+----------------+---------+
|chr1      |817185|817186|G              |[A]             |[NA12878]|
+----------+------+------+---------------+----------------+---------+



In [19]:
spark.sql("select contigName, start, end, referenceAllele, alternateAlleles, genotypes.sampleId[0] from vcf_table \
where genotypes.sampleId[0] = 'NA12878'").show(truncate=False)

+----------+------+------+---------------+----------------+---------------------+
|contigName|start |end   |referenceAllele|alternateAlleles|genotypes.sampleId[0]|
+----------+------+------+---------------+----------------+---------------------+
|chr1      |817185|817186|G              |[A]             |NA12878              |
|chr1      |817340|817341|A              |[G]             |NA12878              |
|chr1      |817888|817889|C              |[G]             |NA12878              |
|chr1      |818801|818802|A              |[G]             |NA12878              |
|chr1      |818811|818812|A              |[G]             |NA12878              |
|chr1      |818953|818954|T              |[C]             |NA12878              |
|chr1      |819122|819123|G              |[A]             |NA12878              |
|chr1      |819583|819584|C              |[T]             |NA12878              |
|chr1      |824319|824320|T              |[C]             |NA12878              |
|chr1      |8244

WARNING	2022-10-07 23:29:38	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream


In [ ]:
# Page Break

# Filtering Somatic or Germline

* Single merged `vcf_table` is great!
* However. Can I still filter, say, I wanted to query Somatic records only or vice versa?
* Recall that `INFO_SOMTYPE` or `INFO_FREQ` columns only present in Somatic VCF header.
* Hence, we can approximate `NULL` record present of either column as pivotal data filter.
* Let try example with `INFO_SOMTYPE` column.

In [20]:
# Count total records
spark.sql("select count(1) as number_of_variants from vcf_table").show()

WARNING	2022-10-07 23:29:46	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:29:46	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:29:46	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:29:46	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:29:48	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream


+------------------+
|number_of_variants|
+------------------+
|           3206719|
+------------------+



In [21]:
# Filter Somatic only records
spark.sql("select count(1) as number_of_variants_somatic from vcf_table where INFO_SOMTYPE is not null").show()

WARNING	2022-10-07 23:29:53	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:29:53	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:29:53	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:29:53	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:29:55	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream


+--------------------------+
|number_of_variants_somatic|
+--------------------------+
|                   1082945|
+--------------------------+



In [22]:
# Filter Germline only records
spark.sql("select count(1) as number_of_variants_germline from vcf_table where INFO_SOMTYPE is null").show()

WARNING	2022-10-07 23:30:01	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:30:01	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:30:01	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:30:01	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2022-10-07 23:30:03	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream


+---------------------------+
|number_of_variants_germline|
+---------------------------+
|                    2123774|
+---------------------------+



# Summary

* We can process VCF by their study type Somatic or Germline calling or both.
* Assumption is that within each "best practice" BioInformatics Pipeline; it should generate similar VCF structure with minor differences in header annotation.
* Couple of strategy possible:
    * by arranging all Somatic VCF type of the same BioInfo Pipeline output into one table
    * similarly, all Germline VCF type of the same BioInfo Pipeline output into one table
    * if we merged VCF, make sure to have very discriminator column that can filter data records better
        * depends on data pipline setup, this discriminator column can be inserted during post-processing VCF files
        * or, could add as part of BioInfo Pipeline VCF annotation process
* We can prescribe Spark/Glow to auto-discover "schema" out of VCF. Hence, "schema evolution" is possible.
* Or, we can prescribe "pre-defined schema" to Spark/Glow during parsing. Hence, enforcing "strict schema".


In [ ]:
# Page Break

# Stop Spark Session

In [23]:
spark.stop()

In [ ]:
# Continue to next notebook